# Solving the Protein Translator problem using F# and FParsec

This article/workbook is my attempt to solve the [Protein Translation Problem](https://exercism.io/tracks/fsharp/exercises/protein-translation) on _exercism.io_.


Last week, I tried this problem using Haskell (you can see the solution [here](https://github.com/tkshill/protein-translation/blob/master/src/ProteinTranslation.hs)) as an exploration of point free style and composition.

So let's now attempt this problem another way, using [FParsec](https://www.quanttec.com/fparsec/), a fully functional **parser combinator** library.

Without getting too much into the details, Parser Combinator libraries operate on the principle of building parsers for large, complex, or intricate data structure, by _combining_ smaller parsers together.

So let's jump in.

## Describing the problem space

(Taken from exercism):

RNA can be broken into three nucleotide sequences called codons, and then translated to a polypeptide like so:

RNA: "AUGUUUUCU" => translates to

Codons: "AUG", "UUU", "UCU" => which become a polypeptide with the following sequence =>

Protein: "Methionine", "Phenylalanine", "Serine"

There are also three terminating codons (also known as 'STOP' codons); if any of these codons are encountered (by the ribosome), all translation ends and the protein is terminated.

All subsequent codons after are ignored, like this:

RNA: "AUGUUUUCUUAAAUG" =>

Codons: "AUG", "UUU", "UCU", "UAA", "AUG" =>

Protein: "Methionine", "Phenylalanine", "Serine"

Note the stop codon "UAA" terminates the translation and the final methionine is not translated into the protein sequence.

Below are the codons and resulting Amino Acids needed for the exercise.


AUG -> Methionine

UUU, UUC -> Phenylalanine

UUA, UUG -> Leucine

UCU, UCC, UCA, UCG -> Serine

UAU, UAC -> Tyrosine

UGU, UGC -> Cysteine

UGG -> Tryptophan

UAA, UAG, UGA -> STOP



## Defining our goal

To call this solution solved, our requirements are as follows:
- Define a function `proteins` that accepts a `string`, and returns an [Option type](https://fsharpforfunandprofit.com/posts/the-option-type/) which either produces a List of Strings representing our proteins or `None`. This is the original goal of the exercise.
- An additional requirement that our solution stop consuming inputs once the appropriate output has been achieved.  

## Importing the relevant libraries

It turns out we only need to import the FParsec library. Everything else we need is a native function.

First let's try to download the appropriate packet

In [1]:
#r "nuget: FParsec"
open FParsec

## Defining our domain

Let's create some types to represent our domain space. In this case, we just need presentation of our codons and our proteins.

We're going to represent our codons as two separate types, `ProteinCodon`s, which can be translated into protein amino acids, and `StopCodon`s, which represent STOP.

In [1]:
type ProteinCodon = AUG | UUU | UUC | UUA | UUG | UCU | UCC | UCA | UCG | UAU | UAC | UGU | UGC | UGG 

type StopCodon = UAA | UAG | UGA

type Protein = Methionine | Phenylalanine | Leucine | Serine | Tyrosine | Cysteine | Tryptophan


## Defining our helper functions

We need a few functions to help us out here. `toProtein` takes a `ProteinCodon` and returns the corresponding `Protein`.

In [1]:
let toProtein =
  function
    | AUG -> Methionine
    | UUU -> Phenylalanine
    | UUC -> Phenylalanine
    | UUA -> Leucine
    | UUG -> Leucine
    | UCU -> Serine
    | UCC -> Serine
    | UCA -> Serine
    | UCG -> Serine
    | UAU -> Tyrosine
    | UAC -> Tyrosine
    | UGU -> Cysteine
    | UGC -> Cysteine
    | UGG -> Tryptophan

// test

toProtein UGG

## Defining our basic parsers

The most complex part of this is our generic `pUnion` parser. Without getting to much into the details, we use the FParsec functions [stringReturn](https://www.quanttec.com/fparsec/reference/charparsers.html#members.stringReturn) and [choice](https://www.quanttec.com/fparsec/reference/primitives.html#members.choice).

`stringReturn` accepts a string to parse against and a type to return and makes a Parser of that type. 

`choice` accepts a sequence of many parsers of the same type and returns a Parser of that type.

Together, they define a parametrized parser that accepts a sum type like one of the codons and makes a parser that will compare a string to see if it's representations.

Thus we can say `pUnion<ProteinCodon>` and get a parser of type `Parser<ProteinCodon, unit>`, or    `pUnion<Protein>` to get a parser of type `Parser<Protein, unit>`

In [1]:
open FSharp.Reflection

let pUnion<'t> : Parser<'t, unit> =
    let pCase (case:UnionCaseInfo) =
        stringReturn case.Name (FSharpValue.MakeUnion(case, [||]) :?> 't)

    FSharpType.GetUnionCases typeof<'t> |> Seq.map pCase |> choice

## Composing our parsers

Now we can start combining some parsers together to get what we want.

In [1]:
let pProteinString = pUnion<ProteinCodon> |>> toProtein |>> fun p -> p.ToString()

`pProteinString` above starts with the parser for protein codons `pUnion<ProteinCocon>` and then uses the `|>>` operator to map the parser results to `toProtein` and that map those value to the `ToString` method. The result is a `Parser` capable of checking if a string matches a `ProteinCodon`, but returns a `string` representing a `Protein`. It's full return type is `Parser<string, unit>`.

Let's see what it tells us with a valid input. The `run` function is what we use to actually execute our parsers on some data.

In [1]:

run pProteinString "AUG"

Item1,Item2,Item3
Methionine,<null>,"(Ln: 1, Col: 4)"


Note that Item1 is our Protein string. Let's now try an invalid input. Something that shouldn't be parsable into a protein.

In [1]:
run pProteinString "UGA"

Item1,Item2,Item3
"Error in Ln: 1 Col: 1 UGA ^ Expecting: 'AUG', 'UAC', 'UAU', 'UCA', 'UCC', 'UCG', 'UCU', 'UGC', 'UGG', 'UGU' , 'UUA', 'UUC', 'UUG' or 'UUU'","Error in Ln: 1 Col: 1 Expecting: 'AUG', 'UAC', 'UAU', 'UCA', 'UCC', 'UCG', 'UCU', 'UGC', 'UGG', 'UGU' , 'UUA', 'UUC', 'UUG' or 'UUU'",<null>


Error! or well, successful error? Out parser successfully rejected the StopCodon string we passed in, and told us it was looking for a string representing a ProteinCodon!

Let's make some more parsers.

In [1]:
let pStopCodon = pUnion<StopCodon> >>% ()

let pStopCodonOrEOL = pStopCodon <|> eof

run pStopCodonOrEOL "UGA" //valid StopCodon input

Item1,Item2,Item3
<null>,<null>,"(Ln: 1, Col: 4)"


In [1]:

run pStopCodonOrEOL "" // valid "end of line" input

Item1,Item2,Item3
<null>,<null>,"(Ln: 1, Col: 1)"


In [1]:

run pStopCodonOrEOL "xxx" //invalid input

Item1,Item2,Item3
"Error in Ln: 1 Col: 1 xxx ^ Expecting: end of input, 'UAA', 'UAG' or 'UGA'","Error in Ln: 1 Col: 1 Expecting: end of input, 'UAA', 'UAG' or 'UGA'",<null>


Success. Our parser successfully accepts a `StopCodon` or the end of a string but rejects anything else. Note that our "unit" type output from the successful parsers is returned as `<null>`. 

Next, we make the parser for our `StopCodon`, `pStopCodon`. Note that `pStopCodon` parses a `StopCodon`, but then uses the `FParsec` operator `>>%` to ignore that value and return a `unit` instead. This is so we can combine it with the `eol` (end of line) parser in `pStopOrEol`. 

`<|>` is another `FParsec` operator which takes two parsers and returns a parser that can be either one, so long as they're the same type. Since `eol` returns a unit `()` we needed to get stop to do the same.

In [1]:
let pProteinStringList = manyTill pProteinString pStopCodonOrEOL

Finally we put it all together in the `pProteinStringList` parser. Here we use the `FParsec` function `manyTill`.

`manyTill` is a function that takes a two parsers and will continue to collect values of the first until it hits a value of the second, and then returns the values from the first as a list. This is conveniently, exactly what we need for this challenge!

## Running our solution

In [1]:
let proteins s =
    match run pProteinStringList s with
    | Success (result, _, _) -> Some result
    | Failure _ -> None

Now we can finally put it all together with our proteins functions which accepts a string and uses the `FParsec` function `run` which is what actually applies our parsers to some string.

`run` returns a `ParserResult` which is a success if the input was parsed without error and a `Failure` with error messages if it doesn't. Here we're choosing to ignore all the extra meta data and just convert these to an `option<list<string>>` type.

In [1]:
proteins "AUG" // returns [ "Methionine" ]

Value
[ Methionine ]


In [1]:
proteins "UGA" // returns [] 

Value
[ ]


In [1]:
proteins "AUGUCUUAGAUG" // returns [ "Methionine", "Serine" ]

Value
"[ Methionine, Serine ]"


Here's the entire solution:

```
#r "nuget: FParsec"
open FParsec

type ProteinCodon = AUG | UUU | UUC | UUA | UUG | UCU | UCC | UCA | UCG | UAU | UAC | UGU | UGC | UGG 

type StopCodon = UAA | UAG | UGA

type Protein = Methionine | Phenylalanine | Leucine | Serine | Tyrosine | Cysteine | Tryptophan

let toProtein =
  function
    | AUG -> Methionine
    | UUU -> Phenylalanine
    | UUC -> Phenylalanine
    | UUA -> Leucine
    | UUG -> Leucine
    | UCU -> Serine
    | UCC -> Serine
    | UCA -> Serine
    | UCG -> Serine
    | UAU -> Tyrosine
    | UAC -> Tyrosine
    | UGU -> Cysteine
    | UGC -> Cysteine
    | UGG -> Tryptophan

let pUnion<'t> : Parser<'t, unit> =
    let pCase (case:UnionCaseInfo) =
        stringReturn case.Name (FSharpValue.MakeUnion(case, [||]) :?> 't)

    FSharpType.GetUnionCases typeof<'t> |> Seq.map pCase |> choice

let pProteinString = pUnion<ProteinCodon> |>> toProtein |>> fun p -> p.ToString()

let pStopCodon = pUnion<StopCodon> >>% ()

let pStopCodonOrEOL = pStopCodon <|> eof

let pProteinStringList = manyTill pProteinString pStopCodonOrEOL

let proteins s =
    match run pProteinStringList s with
    | Success (result, _, _) -> Some result
    | Failure _ -> None
```